# ASEdb-SQLite3 backend example

In [10]:
import numpy as np
from ase.lattice import bulk
from ase.calculators.singlepoint import SinglePointCalculator
from ase.db.row import atoms2dict

from abcd.structurebox import StructureBox
from abcd.authentication import Credentials
import asedb_sqlite3_backend.asedb_sqlite3_backend as asedb_sqlite3_backend

In [36]:
db = 'test.db'

###Establish the connection to the backend, generate random configurations and insert them to the database.

In [37]:
!rm ~/databases/all/test.db
box = StructureBox(asedb_sqlite3_backend.ASEdbSQlite3Backend(database=db, user=None))
token = box.authenticate(Credentials(None))

N = 10
for i in range(N):
    atoms = bulk('Si', crystalstructure='diamond', a=5.43, cubic=True)
    atoms.rattle()

    # simulate a calculation with random results
    e = np.random.uniform()
    f = np.random.uniform(size=3*len(atoms)).reshape((len(atoms), 3))
    s = np.random.uniform(size=9).reshape((3, 3))
    calc = SinglePointCalculator(atoms, energy=e, forces=f, stress=s)
    atoms.set_calculator(calc)
    f = atoms.get_forces()
    e = atoms.get_potential_energy()

    # add some arbitrary data
    atoms.info['integer_info'] = 42
    atoms.info['real_info'] = 217
    atoms.info['config_type'] = 'diamond'
    atoms.new_array('array_data', np.ones_like(atoms.numbers))
    
    box.insert(token, atoms)

### Use ase-db to display the created database

In [38]:
!ase-db ~/databases/all/$db

id|age|user   |formula|calculator|energy| fmax|pbc| volume|charge|   mass| smax
 1| 3s|patrick|Si8    |unknown   | 0.782|1.354|TTT|160.103| 0.000|224.684|0.882
 2| 3s|patrick|Si8    |unknown   | 0.241|1.180|TTT|160.103| 0.000|224.684|0.788
 3| 3s|patrick|Si8    |unknown   | 0.323|1.338|TTT|160.103| 0.000|224.684|0.969
 4| 3s|patrick|Si8    |unknown   | 0.656|1.313|TTT|160.103| 0.000|224.684|0.856
 5| 3s|patrick|Si8    |unknown   | 0.086|1.281|TTT|160.103| 0.000|224.684|0.751
 6| 3s|patrick|Si8    |unknown   | 0.164|1.090|TTT|160.103| 0.000|224.684|0.868
 7| 3s|patrick|Si8    |unknown   | 0.967|1.483|TTT|160.103| 0.000|224.684|0.692
 8| 2s|patrick|Si8    |unknown   | 0.875|1.385|TTT|160.103| 0.000|224.684|0.823
 9| 2s|patrick|Si8    |unknown   | 0.659|1.463|TTT|160.103| 0.000|224.684|0.924
10| 2s|patrick|Si8    |unknown   | 0.737|1.346|TTT|160.103| 0.000|224.684|0.921
Rows: 10
Keys: config_type, integer_info, real_info


###Query the database and get Atoms objects back

In [45]:
count = 0
for i in box.find(token, 'energy>0.5', limit=5, sort='energy'):
    print(i)
    count += 1
print '\nFound', count, 'structures'

Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))
Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))
Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))
Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))
Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))

Found 5 structures


### Copy all atoms with id greater than 7

In [40]:
# Copy all Atoms structures with id greater than 7
atoms_it = box.find(token, 'id>7')
atoms = [at for at in atoms_it]
print 'Copied', len(atoms), 'structures'

Copied 3 structures


### Delete them from the database

In [41]:
res = box.remove(token, 'id>7', just_one=False)
print res.msg

Deleted 3 rows


In [42]:
!ase-db ~/databases/all/test.db -c ++

id|age|user   |formula|calculator|energy| fmax|pbc| volume|charge|   mass| smax|real_info|integer_info|config_type
 1|14s|patrick|Si8    |unknown   | 0.782|1.354|TTT|160.103| 0.000|224.684|0.882|      217|          42|diamond    
 2|14s|patrick|Si8    |unknown   | 0.241|1.180|TTT|160.103| 0.000|224.684|0.788|      217|          42|diamond    
 3|14s|patrick|Si8    |unknown   | 0.323|1.338|TTT|160.103| 0.000|224.684|0.969|      217|          42|diamond    
 4|14s|patrick|Si8    |unknown   | 0.656|1.313|TTT|160.103| 0.000|224.684|0.856|      217|          42|diamond    
 5|14s|patrick|Si8    |unknown   | 0.086|1.281|TTT|160.103| 0.000|224.684|0.751|      217|          42|diamond    
 6|14s|patrick|Si8    |unknown   | 0.164|1.090|TTT|160.103| 0.000|224.684|0.868|      217|          42|diamond    
 7|14s|patrick|Si8    |unknown   | 0.967|1.483|TTT|160.103| 0.000|224.684|0.692|      217|          42|diamond    
Rows: 7
Keys: config_type, integer_info, real_info


### Insert the copies back into the database

In [43]:
res = box.insert(token, atoms)
print res.msg, 'at ids', res.inserted_ids

Inserted 3 configurations at ids [11, 12, 13]


In [46]:
!ase-db ~/databases/all/test.db -c ++

id|age|user   |formula|calculator|energy| fmax|pbc| volume|charge|   mass| smax|real_info|integer_info|config_type
 1|25s|patrick|Si8    |unknown   | 0.782|1.354|TTT|160.103| 0.000|224.684|0.882|      217|          42|diamond    
 2|25s|patrick|Si8    |unknown   | 0.241|1.180|TTT|160.103| 0.000|224.684|0.788|      217|          42|diamond    
 3|25s|patrick|Si8    |unknown   | 0.323|1.338|TTT|160.103| 0.000|224.684|0.969|      217|          42|diamond    
 4|25s|patrick|Si8    |unknown   | 0.656|1.313|TTT|160.103| 0.000|224.684|0.856|      217|          42|diamond    
 5|25s|patrick|Si8    |unknown   | 0.086|1.281|TTT|160.103| 0.000|224.684|0.751|      217|          42|diamond    
 6|25s|patrick|Si8    |unknown   | 0.164|1.090|TTT|160.103| 0.000|224.684|0.868|      217|          42|diamond    
 7|25s|patrick|Si8    |unknown   | 0.967|1.483|TTT|160.103| 0.000|224.684|0.692|      217|          42|diamond    
11|10s|patrick|Si8    |unknown   | 0.875|1.385|TTT|160.103| 0.000|224.684|0.823|